In [3]:
import cv2
import os
import string

img = cv2.imread("/content/mypic.jpg") # Replace with the correct image path#image

msg = input("Enter secret message:")
password = input("Enter a passcode:")

d = {}
c = {}

for i in range(255):
    d[chr(i)] = i
    c[i] = chr(i)

m = 0
n = 0
z = 0

for i in range(len(msg)):
    img[n, m, z] = d[msg[i]]
    n = n + 1
    m = m + 1
    z = (z + 1) % 3

cv2.imwrite("encryptedImage.jpg", img)
os.system("start encryptedImage.jpg")  # Use 'start' to open the image on Windows

message = ""
n = 0
m = 0
z = 0

pas = input("Enter passcode for Decryption")
if password == pas:
    for i in range(len(msg)):
        message = message + c[img[n, m, z]]
        n = n + 1
        m = m + 1
        z = (z + 1) % 3
    print("Decryption message:", message)
else:
    print("YOU ARE NOT auth")

Enter secret message:1234
Enter a passcode:456
Enter passcode for Decryption456
Decryption message: 1234


In [6]:
!pip install pycrypto
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.5 MB/s eta 0:00:00


In [19]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import cv2
import os
import base64
import hashlib
import numpy as np

# Function to encrypt message
def encrypt_message(message, key):
    key = hashlib.sha256(key.encode()).digest()  # Hash the key to ensure 256-bit length
    cipher = AES.new(key, AES.MODE_CBC)
    encrypted_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    return base64.b64encode(cipher.iv + encrypted_bytes).decode()

# Function to decrypt message
def decrypt_message(encrypted_message, key):
    key = hashlib.sha256(key.encode()).digest()
    encrypted_bytes = base64.b64decode(encrypted_message)
    iv = encrypted_bytes[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(encrypted_bytes[16:]), AES.block_size).decode()

# Hash password for storage
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Function to embed data using LSB
def embed_data(img, data):
    data += "END"  # Adding an end marker to detect message end
    binary_data = ''.join(format(ord(c), '08b') for c in data)
    data_index = 0

    for row in img:
        for pixel in row:
            for i in range(3):  # Modify LSB of each color channel
                if data_index < len(binary_data):
                    pixel[i] = (pixel[i] & ~1) | int(binary_data[data_index])
                    data_index += 1
                else:
                    return img
    return img

# Function to extract data from LSB
def extract_data(img):
    binary_data = ""
    for row in img:
        for pixel in row:
            for i in range(3):
                binary_data += str(pixel[i] & 1)

    message = ""
    for i in range(0, len(binary_data), 8):
        char = chr(int(binary_data[i:i+8], 2))
        if message[-3:] == "END":
            return message[:-3]
        message += char
    return message

# Load image
img = cv2.imread("mypic.png")  # Replace with actual image path

msg = input("Enter secret message: ")
password = input("Enter a passcode: ")
hashed_password = hash_password(password)

# Encrypt the message
encrypted_msg = encrypt_message(msg, password)

# Encode encrypted message into the image using LSB
img = embed_data(img, encrypted_msg)
cv2.imwrite("encryptedImage.png", img)
os.system("start encryptedImage.png")

# Decryption phase
pas = input("Enter passcode for decryption: ")
if hash_password(pas) == hashed_password:
    extracted_data = extract_data(img)
    try:
        original_message = decrypt_message(extracted_data, pas)
        print("Decryption successful: ", original_message)
    except:
        print("Error: Incorrect password or corrupted data.")
else:
    print("YOU ARE NOT AUTHORIZED")


Enter secret message: hello how are you
Enter a passcode: 33344
Enter passcode for decryption: 33344
Decryption successful:  hello how are you
